In [9]:
from scraper import WebScraper

In [3]:
foldername = "Results_from_scraper"

def start_scraping(keywords:list, sites:dict):
    for keyword in keywords:
        for site in sites:
            scraper = WebScraper([keyword])
            scraper.scrape(sites[site])
            scraper.save_results_to_file(filename= "scraped_data", folder= foldername, filetype= "json", append_existing_file= True)
            scraper.save_html(filename= "htmlcontents", folder= foldername)
            print()

# Bewerbung
### Wo kann ich mich bewerben und welche Voraussetzungen müssen erfüllt sein? 

In [ ]:
application_keywords = ["bewerben", "Bewerbung", "Voraussetzung"]

application_sites = {
    "hwr" : "https://www.hwr-berlin.de/studium/bewerbung/bewerbungsverfahren/#c25999",
    "anabin" : "https://anabin.kmk.org/no_cache/filter/schulabschluesse-mit-hochschulzugang.html#land_gewaehlt",
    "daad" : "https://www.daad.de/de/in-deutschland-studieren/voraussetzungen/zulassungsdatenbank/",
    "berlinde": "https://www.berlin.de/sen/wissenschaft/studium/hochschulzulassung/#:~:text=Nicht-EU-Ausl%C3%A4nderinnen%20und%20Ausl%C3%A4nder%2C%20die%20kein%20Abitur%20und%20keinen,studieren%2C%20wenn%20sie%20einen%20gleichwertigen%20ausl%C3%A4ndischen%20Schulabschluss%20nachweisen.",
    "hochschulstart" : "https://www.hochschulstart.de/informieren-planen/internationale-bewerbende",
    "htw-bewerbung" : "https://www.htw-berlin.de/studium/bewerbung/bewerbung-ueber-uni-assist/",
    "hu": "https://www.hu-berlin.de/de/studium/bewerbung/bewerbung-und-zulassung/informationen-fuer-internationale-studieninteressierte/informationen-zur-bewerbung-fuer-internationale-studieninteressierte",
    "uni-assist" : "https://www.uni-assist.de/tools/uni-assist-hochschulen/",
    "fu-berlin" : "https://www.fu-berlin.de/studium/international/studium_fu/index.html",
    "htw-bewerbung2" : "https://www.htw-berlin.de/studium/bewerbung/bewerbung-ueber-uni-assist/zeugnisbewertung/",
    "fu-berlin2" : "https://www.fu-berlin.de/studium/bewerbung/bachelor/ausl-hzb/index.html",
    "tu-berlin" : "https://www.tu.berlin/studierendensekretariat/bewerbung-einschreibung-bachelor/studieninteressierte-mit-auslaendischen-bildungsnachweisen/bewerbung/bewerbungsgruppe-c",
    "hrk" : "https://www.hrk.de/themen/internationales/internationale-studierende-und-forschende/hochschulzugang-fuer-internationale-studierende/uni-assist-ev/"
}

start_scraping(application_keywords, application_sites)

### Welche Sprachkenntnisse muss ich vorweisen und wie werden diese nachgewiesen?

In [ ]:
language_keywords = ["Sprach", "Nachweis"]

languages_sites = {
    "kmk" : "https://www.kmk.org/themen/deutsches-sprachdiplom-dsd/deutsche-sprachkenntnisse-fuer-den-hochschulzugang.html",
    "hochschulstart" : "https://www.hochschulstart.de/informieren-planen/internationale-bewerbende#c773", 
    "hrk" : "https://www.hrk.de/themen/internationales/internationale-studierende-und-forschende/hochschulzugang-fuer-internationale-studierende/sprachnachweis-deutsch/"

}
start_scraping(language_keywords, languages_sites)

### Welche Unterlagen werden für eine Bewerbung benötigt?

In [ ]:
documents_keywords = ["Unterlagen", "Dokumente"]
documents_sites = {
    "daad" : "https://www.daad.de/de/in-deutschland-studieren/voraussetzungen/ueberblick/",
    "uni-assist" : "https://www.uni-assist.de/bewerben/dokumente-sammeln/sonstige-dokumente/",
    "htw-berlin" : "https://www.htw-berlin.de/studium/bewerbung/bewerbung-ueber-uni-assist/#:~:text=Sprachnachweise%2C,sowie%20einen%20verifizierten%20Notenspiegel%20ein.",
    "uni-assist2" : "https://www.uni-assist.de/bewerben/dokumente-sammeln/"
}
start_scraping(documents_keywords, documents_sites)

### Wo kann ich meine Unterlagen beglaubigen lassen?

In [ ]:
authenticate_keywords = ["Beglaubigen", "Notar", "Original"]
authenticate_sites = {
    "auswärtiges-amt" : "https://www.auswaertiges-amt.de/de/service/fragenkatalog-node/-/606802?openAccordionId=item-606188-7-panel",
    "tu-berlin" : "https://www.tu.berlin/studierendensekretariat/themen-a-z/amtliche-beglaubigung-und-uebersetzung",
    "uni-assist" : "https://www.uni-assist.de/bewerben/dokumente-sammeln/beglaubigungen-uebersetzungen/"
}

start_scraping(authenticate_keywords, authenticate_sites)

### Welche Fristen muss ich bei der Bewerbung beachten? 

In [ ]:
application_deadline_keywords = ["Bewerbungsfrist", "Frist", "Termin"]
application_deadline_sites = {
    "int-studierende" : "https://www.internationale-studierende.de/fragen-zur-vorbereitung/studienvorraussetzungen",
    "hu-berlin" : "https://www.hu-berlin.de/de/studium/bewerbung/fristen",
    "fu-berlin" : "https://www.fu-berlin.de/studium/bewerbung/bewerbungsfristen/index.html",
    "tu-berlin" : "https://www.tu.berlin/international/studierende/internationale-studierende/austausch-an-die-tu-berlin/bewerbung-austausch/fristen-und-termine"
}

start_scraping(application_deadline_keywords, application_deadline_sites)

### Wer entscheidet über meine Zulassung zum Studium?

In [ ]:
admission_keywords = ["Zulassung", "entscheid"]
admission_sites = {
    
}

start_scraping(admission_keywords, admission_sites)

### Gibt es einen Eignungstest für die Studiengänge an deutschen Hochschulen?

In [ ]:
aptitude_test_keywords = ["Eignungstest"]
aptitude_test_sites = {
    "hrk" : "https://www.hrk.de/fileadmin/_migrated/content_uploads/Testas-Handreichung-16.2.2011.pdf",
    "hrk2" : "https://www.hrk.de/themen/internationales/internationale-studierende-und-forschende/hochschulzugang-fuer-internationale-studierende/testas/",
    "hrk3" : "https://www.hrk.de/themen/internationales/internationale-studierende-und-forschende/hochschulzugang-fuer-internationale-studierende/"
}

start_scraping(aptitude_test_keywords, aptitude_test_sites)

### Wo bekomme ich weitere Beratungsmöglichkeiten? 

In [ ]:
consulting_keywords = ["Beratung"]
consulting_sites = {
    "tu-berlin" : "https://www.tu.berlin/international/starten-an-der-tu-berlin/services-und-angebote/beratungsangebote-fuer-studierende",
    "studierendenwerk" : "https://www.stw.berlin/international/welcome-to-berlin.html",
    "fu-berlin" : "https://www.fu-berlin.de/einrichtungen/service/studierende/international/index.html"
}

start_scraping(consulting_keywords, consulting_sites)

### Wer entscheidet über die Anerkennung von im Ausland erworbenen Studienleistungen? 

In [ ]:
abroad_keywords = ["Leistungen", "Anerkennung", "Ausland"]
abroad_sites = {
    "kmk" : "https://www.kmk.org/zab/zentralstelle-fuer-auslaendisches-bildungswesen.html",
    "anabin" : "https://anabin.kmk.org/no_cache/filter/hochschulabschluesse.html",
    "anerkennung" : "https://www.anerkennung-in-deutschland.de/html/de/hochschule.php#module9764"
}

start_scraping(abroad_keywords, abroad_sites)

# Leben und Wohnen in Berlin Wohnungen finden 

### Wie finde ich eine Wohnung in Berlin?

In [ ]:
appartment_keywords = ["Wohnung", "Suche", "Wohngemeinschaft", "WG"]
appartment_sites = {
    "hu-berlin": "https://www.international.hu-berlin.de/de/an-die-hu/services-informationen/wohnen",
    "allaboutberlin": "https://allaboutberlin.com/guides/find-a-flat-in-berlin",
    "housinganywhere": "https://housinganywhere.com/de/s/Berlin--Deutschland",
    "immoscout": "https://www.immobilienscout24.de/Suche/de/berlin/berlin/wg-zimmer",
    "wg": "https://www.wg-gesucht.de/wg-zimmer-in-Berlin.8.0.1.0.html",
    "htw-berlin": "https://www.htw-berlin.de/international/wege-an-die-htw-berlin/international-degree-seeking-students/leben-in-berlin/"
}

start_scraping(appartment_keywords, appartment_sites)

### Wie lerne ich Menschen in Berlin am besten kennen?

In [ ]:
get_to_know_keywords = ["kennenlernen"]
get_to_know_sites = {
    "allaboutberlin" : "https://allaboutberlin.com/guides/making-friends",
    "stw" : "https://www.stw.berlin/kultur/get-together/get-together/"

}

start_scraping(get_to_know_keywords, get_to_know_sites)

### Mit welchen Kosten muss ich als Student in Berlin rechnen?

In [10]:
costs_keywords = ["Kosten", "Ausgaben"]  # Stichwörter für die Suche
costs_sites = {
    "fu-berlin" : "https://www.fu-berlin.de/studium/international/studium_fu/studienfinanzierung/index.html",
    "hwr" : "https://www.hwr-berlin.de/studium/international-studieren/international-students/",
    "int-studierende" : "https://www.internationale-studierende.de/finanzierung"
}

start_scraping(costs_keywords, costs_sites)



### Welche Voraussetzungen müssen internationale Studierende erfüllen, um BAföG in Deutschland zu erhalten?

In [ ]:
bafoeg_keywords = ["BAföG", "Voraussetzungen", "Bedingungen"]
bafoeg_sites = {
    "stw" : "https://www.stw.berlin/finanzierung/bafög-faq/bafoeg-international.html",
    "fu-berlin" : "https://www.fu-berlin.de/sites/welcome/studienfinanzierung/BAfoeG/index.html",
    "study-in-germany" : "https://www.study-in-germany.de/de/studium-planen/besondere-lebensumstaende/studienangebote-fuer-gefluechtete/#tab-0b969685",
    "stw2" : "https://www.stw.berlin/finanzierung/"
}

start_scraping(bafoeg_keywords, bafoeg_sites)

### Benötigen internationale Studierende in Deutschland ein Sperrkonto, um ein Studium aufzunehmen?

In [ ]:
blocked_account_keywords = ["Sperrkonto"]
blocked_account_sites = {
    "auswärtiges-amt" : "https://www.auswaertiges-amt.de/de/sperrkonto/375488",
    "studying-in-germany" : "https://www.studying-in-germany.org/de/sperrkonto-auslaendische-studenten/",
    "sparkasse" : "https://www.sparkasse.de/pk/ratgeber/bildung/studium/sperrkonto.html",
    "berlin.de" : "https://www.berlin.de/einwanderung/service/faq/"
}

start_scraping(blocked_account_keywords, blocked_account_sites)

### Welche Stipendienprogramme stehen speziell für internationale Studierende in Berlin zur verfügung?

In [ ]:
scholarship_keywords = ["Stipendium"]
scholarship_sites = {
    "udk" : "https://www.udk-berlin.de/service/international-office/stipendienprogramme/stipendien-fuer-internationale-studierende-an-der-udk-berlin/studienabschluss-stipendien-fuer-internationale-studierende-stibet-i/",
    "udk2" : "https://www.udk-berlin.de/service/international-office/stipendienprogramme/stipendien-fuer-internationale-studierende-an-der-udk-berlin/stipendien-fuer-mehr-chancengerechtigkeit/",
    "mystipendium" : "https://www.mystipendium.de/stipendien/hu-berlin-abschlussstipendien"
}

start_scraping(scholarship_keywords, scholarship_sites)

### Können internationale Studierende während ihres Studiums in Deutschland arbeiten, und wenn ja, wie viele Stunden pro Woche?

In [ ]:
work_keywords = ["Arbeiten", "Lohn", "Gehalt", "Verdienst", "verdienen", "Stunden"]
work_sites = {
    "lmu" : "https://www.lmu.de/de/workspace-fuer-studierende/international-student-guide/arbeiten-in-deutschland/index.html",
    "int-studierende" : "https://www.internationale-studierende.de/waehrend-des-studiums/jobben",
    "daad" : "https://www.daad.de/de/in-deutschland-studieren/arbeit-karriere/nebenjobs/"
}

start_scraping(work_keywords, work_sites)

### Muss ein internationaler Studierender in Deutschland Steuern zahlen?

In [ ]:
tax_keywords = ["Steuern"]
tax_sites = {
    "daad" : "https://www.daad.de/de/",
    "stud" : "https://studentensteuererklaerung.de",
    "arbeitsagentur" : "https://www.arbeitsagentur.de"
}

start_scraping(tax_keywords, tax_sites)

# Aufenthalt:
### Welche Visumstypen benötige ich für ein Studium in Deutschland?
inklusive allgeine Themen rund ums Visum

In [ ]:
visa_keywords = ["Visum", "Voraussetzung", "Antrag", "Verlängerung", "Dauer"]
visa_sites = {
    "germany" : "https://www.germany.info/us-en/service/02-VisaEinreise/00-visa/928242",
    "auswertigesamt" : "https://www.auswaertiges-amt.de/de/service/visa",
    "arbeitsagentur" : "https://www.arbeitsagentur.de/vor-ort/internationales/aufenthaltsgenehmigung",
    "stw" : "https://www.studierendenwerke.de/de/informationen-zum-aufenthalt-und-visa",
    "study-in" : "https://www.study-in-germany.de/de/studium-planen/voraussetzungen/visum/",
    "axa" : "https://www.axa-schengen.com/de/deutschland-visum/studentenvisum-deutschland",
    "bamf" : "https://www.bamf.de/DE/Themen/MigrationAufenthalt/ZuwandererDrittstaaten/Bildung/Studium/studium-node.html",
    "make-it" : "https://www.make-it-in-germany.com/de/",
    "germany-visa" : "https://www.germany-visa.org/student-visa/",
    "axa2" : "https://www.axa-schengen.com/de",
    "germany-visa2" : "https://www.germany-visa.org/student-residence-permit",
    "daad" : "https://www.daad.de/de/in-deutschland-studieren/leben-in-deutschland/visum/",
    "germany-visa3" : "https://www.germany-visa.org/de/studentenvisum/studentenvisum-deutschland-wie-erhalte-ich-ein-deutsches-studentenvisum/",
    "study-in2" : "https://www.studying-in-germany.org/de/studentenvisum-deutschland/",
    "koeln" : "https://www.th-koeln.de/internationales/visum_66057.php",
    "gesetze" : "https://www.gesetze-im-internet.de/aufenthg_2004/__16b.html",
    "ilmenau" : "https://www.tu-ilmenau.de/en/study/in-study/course-of-studies/administrative-guide/visa-and-residence-permit",
    "bamf2" : "https://www.bamf.de/EN/Themen/MigrationAufenthalt/ZuwandererDrittstaaten/Bildung/Studium/studium-node.html"
}

start_scraping(visa_keywords, visa_sites)